In [1]:
# == Hyperparameter configuration ==

# Official scored labels Physionet 2021: https://github.com/physionetchallenges/evaluation-2021/blob/main/dx_mapping_scored.csv

# 0 = 426783006 -> sinus rhythm (SR)
# 1 = 164889003 -> atrial fibrillation (AF)
# 2 = 164890007 -> atrial flutter (AFL)
# 3 = 284470004 or 63593006 -> premature atrial contraction (PAC) or supraventricular premature beats (SVPB)
# 4 = 427172004 or 17338001 -> premature ventricular contractions (PVC), ventricular premature beats (VPB)
# 5 = 6374002 -> bundle branch block (BBB)
# 6 = 426627000 -> bradycardia (Brady)
# 7 = 733534002 or 164909002 -> complete left bundle branch block (CLBBB), left bundle branch block (LBBB)
# 8 = 713427006 or 59118001 -> complete right bundle branch block (CRBBB), right bundle branch block (RBBB)
# 9 = 270492004 -> 1st degree av block (IAVB)
# 10 = 713426002 -> incomplete right bundle branch block (IRBBB)
# 11 = 39732003 -> left axis deviation (LAD)
# 12 = 445118002 -> left anterior fascicular block (LAnFB)
# 13 = 251146004 -> low qrs voltages (LQRSV)
# 14 = 698252002 -> nonspecific intraventricular conduction disorder (NSIVCB)
# 15 = 10370003 -> pacing rhythm (PR)
# 16 = 365413008 -> poor R wave Progression (PRWP)
# 17 = 164947007 -> prolonged pr interval (LPR)
# 18 = 111975006 -> prolonged qt interval (LQT)
# 19 = 164917005 -> qwave abnormal (QAb)
# 20 = 47665007 -> right axis deviation (RAD)
# 21 = 427393009 -> sinus arrhythmia (SA)
# 22 = 426177001 -> sinus bradycardia (SB)
# 23 = 427084000 -> sinus tachycardia (STach)
# 24 = 164934002 -> t wave abnormal (TAb)
# 25 = 59931005 -> t wave inversion (TInv)

VALID_LABELS = set(
    [
        "164889003",
        "164890007",
        "6374002",
        "426627000",
        "733534002",
        "713427006",
        "270492004",
        "713426002",
        "39732003",
        "445118002",
        "164909002",
        "251146004",
        "698252002",
        "426783006",
        "284470004",
        "10370003",
        "365413008",
        "427172004",
        "164947007",
        "111975006",
        "164917005",
        "47665007",
        "59118001",
        "427393009",
        "426177001",
        "427084000",
        "63593006",
        "164934002",
        "59931005",
        "17338001",
    ]
)
# VALID_LABELS = set(["426783006", "164889003", "164890007", "284470004", "427172004"]) # SR, AF, AFL, PAC, PVC
NUM_CLASSES =  26

CLASS_BALANCE = 6000 # imblearn undersampling & imblearn oversampling
TEST_BALANCE = 100 # imblearn undersampling
TRAIN_TEST_SPLIT = 0.8
VALIDATION_SPLIT = 0.25

EPOCHS = 500
LEARNING_RATE = 0.001
BATCH_SIZE = 128

In [2]:
# == Check if GPU is available ==

!nvidia-smi

Thu Jul 18 20:22:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# == Install requirements ==

!pip install google-colab
!pip install tensorflow keras numpy
!pip install h5py tqdm
!pip install pandas scipy imblearn
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.8 MB/s eta 0:00:00


In [4]:
# == Import requirements ==

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
import os
import h5py
from tqdm import tqdm
import pandas as pd
from collections import Counter

import random
import scipy
from scipy.signal import butter, lfilter
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

import matplotlib.pyplot as plt

In [5]:
# == Preprocess functions ==

def pad_or_truncate_ecg(ecg: list, max_samples: int) -> list:
    try:
        padded_or_truncated_ecg = ecg[:max_samples] + [0] * (max_samples - len(ecg))
    except Exception as e:
        print("Fail: padding", e)
    return padded_or_truncated_ecg

def resample_ecg(ecg: list, resample: int):
    new_ecg = scipy.signal.resample(
        ecg, resample, t=None, axis=0, window=None, domain="time"
    )
    return list(new_ecg)

def normalize_to_minus11(ecg: list):
    max_val = max(ecg)
    min_val = min(ecg)
    # Handle the case where max_val and min_val are the same (to avoid division by zero)
    if max_val == min_val:
        return [0 for _ in ecg]
    normalized_values = [2 * (x - min_val) / (max_val - min_val) - 1 for x in ecg]
    return normalized_values

def butter_bandpass(lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype="band")
    return b, a

def butter_bandpass_filter(ecg: list, lowcut: float, highcut: float, sampling_rate: int, order: int =4):
    b, a = butter_bandpass(lowcut, highcut, sampling_rate, order=order)
    y = lfilter(b, a, ecg)
    return y

def split_list_into_n_sublists(lst, n):
    k, m = divmod(len(lst), n)
    return [lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)]

In [6]:
# == Map labels to numerical values functions ==

# Official scored labels Physionet 2021: https://github.com/physionetchallenges/evaluation-2021/blob/main/dx_mapping_scored.csv

arrhyhtmia_mapping_id_to_index = {
    "426783006": 0, # sinus rhythm (SR)
    "164889003": 1, # atrial fibrillation (AF)
    "164890007": 2, # atrial flutter (AFL)
    "284470004": 3, # premature atrial contraction (PAC)
    "63593006": 3, # supraventricular premature beats (SVPB)
    "427172004": 4, # premature ventricular contractions (PVC)
    "17338001": 4, # ventricular premature beats (VPB)
    "6374002": 5, # bundle branch block (BBB)
    "426627000": 6, # bradycardia (Brady)
    "733534002": 7, # complete left bundle branch block (CLBBB)
    "164909002": 7, # left bundle branch block (LBBB)
    "713427006": 8, # complete right bundle branch block (CRBBB)
    "59118001": 8, # right bundle branch block (RBBB)
    "270492004": 9, # 1st degree av block (IAVB)
    "713426002": 10, # incomplete right bundle branch block (IRBBB)
    "39732003": 11, # left axis deviation (LAD)
    "445118002": 12, # left anterior fascicular block (LAnFB)
    "251146004": 13, # low qrs voltages (LQRSV)
    "698252002": 14, # nonspecific intraventricular conduction disorder (NSIVCB)
    "10370003": 15, # pacing rhythm (PR)
    "365413008": 16, # poor R wave Progression (PRWP)
    "164947007": 17, # prolonged pr interval (LPR)
    "111975006": 18, # prolonged qt interval (LQT)
    "164917005": 19, # qwave abnormal (QAb)
    "47665007": 20,  # right axis deviation (RAD)
    "427393009": 21, # sinus arrhythmia (SA)
    "426177001": 22, # sinus bradycardia (SB)
    "427084000": 23, # sinus tachycardia (STach)
    "164934002": 24, # t wave abnormal (TAb)
    "59931005": 25 # t wave inversion (TInv)
}

def map_arrhyhtmia_id_to_index(x: str) -> int:
    return arrhyhtmia_mapping_id_to_index[x]

arrhyhtmia_mapping_index_to_id = {
    0: "426783006", # sinus rhythm (SR)
    1: "164889003", # atrial fibrillation (AF)
    2: "164890007", # atrial flutter (AFL)
    3: "284470004|63593006", # premature atrial contraction (PAC) | supraventricular premature beats (SVPB)
    4: "427172004|17338001", # premature ventricular contractions (PVC) | ventricular premature beats (VPB)
    5: "6374002", # bundle branch block (BBB)
    6: "426627000", # bradycardia (Brady)
    7: "733534002|164909002", # complete left bundle branch block (CLBBB) | left bundle branch block (LBBB)
    8: "713427006|59118001", # complete right bundle branch block (CRBBB) | right bundle branch block (RBBB)
    9: "270492004", # 1st degree av block (IAVB)
    10: "713426002", # incomplete right bundle branch block (IRBBB)
    11: "39732003", # left axis deviation (LAD)
    12: "445118002", # left anterior fascicular block (LAnFB)
    13: "251146004", # low qrs voltages (LQRSV)
    14: "698252002", # nonspecific intraventricular conduction disorder (NSIVCB)
    15: "10370003", # pacing rhythm (PR)
    16: "365413008", # poor R wave Progression (PRWP)
    17: "164947007", # prolonged pr interval (LPR)
    18: "111975006", # prolonged qt interval (LQT)
    19: "164917005", # qwave abnormal (QAb)
    20: "47665007",  # right axis deviation (RAD)
    21: "427393009", # sinus arrhythmia (SA)
    22: "426177001", # sinus bradycardia (SB)
    23: "427084000", # sinus tachycardia (STach)
    24: "164934002", # t wave abnormal (TAb)
    25: "59931005" # t wave inversion (TInv)
}

def map_arrhyhtmia_index_to_id(x: int) -> str:
    return arrhyhtmia_mapping_index_to_id[x]

In [7]:
# == Mount drive ==

# https://drive.google.com/drive/folders/1L_gOMrkygu2N0k97COYuVrmE-AwEEMoQ

drive.mount('/content/drive')
path = "/content/drive/My Drive/Master Thesis/Datasets"
!ls "/content/drive/My Drive/Master Thesis/Datasets"

Mounted at /content/drive
codes_SNOMED.csv  physionet2017_references.csv	physionet2021_references.csv
physionet2017.h5  physionet2021.h5		prepared


In [ ]:
# == Load all Physionet2021 ECGs and their IDs to a dictionary X_dict ==

X_dict = {}
Y_dict = {}

h5file = h5py.File(os.path.join(path, "prepared/physionet2021_scoredLabels.h5"), "r")
IDs = list(h5file.keys())
pbar = tqdm(total=len(IDs), desc="Load ECG data", position=0, leave=True)
for key in IDs:
    X_dict[key] = list(h5file[key][0])
    pbar.update(1)

# == Load all labels and their IDs to a dictionary Y_dict (some ECGs can have multiple labels) ==

labels_df = pd.read_csv(os.path.join(path, "physionet2021_references.csv"), sep=";")
pbar = tqdm(total=len(labels_df), desc="Load ECG labels", position=0, leave=True)
for _, row in labels_df.iterrows():
    labels = row["labels"].strip().split(",")
    labels_valid = []
    for label in labels:
        if label in VALID_LABELS:
              labels_valid.append(map_arrhyhtmia_id_to_index(label))
    if row["id"] in X_dict:
        Y_dict[row["id"]] = labels_valid
    pbar.update(1)

# del IDs, h5file

Load ECG data:  67%|██████▋   | 54629/81960 [02:09<03:24, 133.67it/s]

In [ ]:
# == Preprocess ECGs ==

pbar = tqdm(total=len(X_dict), desc="Preprocess ECGs", position=0, leave=True)
for key in X_dict:
    X_dict[key] = pad_or_truncate_ecg(ecg=X_dict[key], max_samples=5000)
    X_dict[key] = resample_ecg(ecg=X_dict[key], resample=2000)
    X_dict[key] = normalize_to_minus11(ecg=X_dict[key])
    X_dict[key] = butter_bandpass_filter(ecg=X_dict[key], lowcut=0.3, highcut=21.0, sampling_rate=200)
    pbar.update(1)


In [ ]:
# == Map scored labels to ECGs and create three lists (X: ECGs, Y: labels, Z: IDs) ==

Y = []
Z = []

for patient_id in tqdm(Y_dict, desc="Map labels to ECGs", position=0, leave=True):
    for label in Y_dict[patient_id]:
        Y.append(label)
        Z.append(str(patient_id))

In [ ]:
# == Split train-test sets by patients if train-test splits files are given ==

if os.path.exists(os.path.join(path, "prepared/trainset_patient_ids.txt")) and os.path.exists(os.path.join(path, "prepared/testset_patient_ids.txt")):
    f = open(os.path.join(path, "prepared/trainset_patient_ids.txt"), "r", encoding="utf-8")
    trainset_patient_ids = f.readlines()
    f.close()
    f = open(os.path.join(path, "prepared/testset_patient_ids.txt"), "r", encoding="utf-8")
    testset_patient_ids = f.readlines()
    f.close()
    trainset_patient_ids = list(map(lambda x: x.replace("\n", ""), trainset_patient_ids))
    testset_patient_ids = list(map(lambda x: x.replace("\n", ""), testset_patient_ids))
    trainset_patient_ids = set(trainset_patient_ids)
    testset_patient_ids = set(testset_patient_ids)
    Y_train = []
    Z_train = []
    Y_test = []
    Z_test = []
    for sample in zip(Y, Z):
        if sample[1] in trainset_patient_ids:
            Y_train.append(sample[0])
            Z_train.append(sample[1])
        elif sample[1] in testset_patient_ids:
            Y_test.append(sample[0])
            Z_test.append(sample[1])
    # del Y, Z
else:
    print("No predefined train-test split files are found!")

In [ ]:
# == Split data train-test sets by patients ==

if not os.path.exists(os.path.join(path, "prepared/trainset_patient_ids.txt")) or not os.path.exists(os.path.join(path, "prepared/testset_patient_ids.txt")):
    bins = []
    for x in range(NUM_CLASSES):
        bins.append([])

    # Create num_classes bins (1 bin = 1 class)
    for sample in zip(Y, Z):
        bins[sample[0]].append([sample[0], sample[1]])

    # Create train-test bins (e.g. TRAIN_TEST_SPLIT = 80%-20%)
    train_bins = []
    test_bins = []
    for x in range(NUM_CLASSES):
        train_bins.append([])
        test_bins.append([])
    for index, bin in enumerate(bins):
        split_index = int(len(bin) * TRAIN_TEST_SPLIT)
        train_bins[index] = bin[:split_index]
        test_bins[index] = bin[split_index:]

    # Create test set (sort bins by occurence of labels descending before adding to train set because of minor classes and multiple labels and add if patient id is not in train set)
    Y_test = []
    Z_test = []
    id_already_in_x_test = set()
    test_bins.sort(key=lambda x: len(x))
    for bin in test_bins:
        for sample in bin:
            id_already_in_x_test.add(sample[1])
            Y_test.append(sample[0])
            Z_test.append(sample[1])

    # Create train set (sort bins by occurence of labels descending before adding to train set because of minor classes and multiple labels and add if patient id is not in train (split by patients) or test set (multiple labels in test set would bias the evaluation) already)
    Y_train = []
    Z_train = []
    id_already_in_x_train = set()
    train_bins.sort(key=lambda x: len(x))
    for bin in train_bins:
        for sample in bin:
            if sample[1] not in id_already_in_x_test:
                id_already_in_x_train.add(sample[1])
                Y_train.append(sample[0])
                Z_train.append(sample[1])

    # Write train and test patient ids to file
    id_already_in_x_train_list = list(id_already_in_x_train)
    id_already_in_x_train_list = list(map(lambda x: str(x) + "\n", id_already_in_x_train_list))
    id_already_in_x_test_list = list(id_already_in_x_test)
    id_already_in_x_test_list = list(map(lambda x: str(x) + "\n", id_already_in_x_test_list))
    # with open('trainset_patient_ids.txt', 'w', encoding='utf-8') as file:
    #    file.writelines(id_already_in_x_train_list)
    # with open('testset_patient_ids.txt', 'w', encoding='utf-8') as file:
    #    file.writelines(id_already_in_x_test_list)
    # del Y, Z
else:
    print("Train-test split files are present, no new split was created!")

In [ ]:
for index, z in enumerate(Z_train):
    Z_train[index] = [Z_train[index]]
for index, z in enumerate(Z_test):
    Z_test[index] = [Z_test[index]]

In [ ]:
# == Balance data ==

# Trainset

count_train = Counter(Y_train)

undersampling_strategy = {}
oversampling_strategy = {}
for i in count_train:
    if count_train[i] > CLASS_BALANCE:
        undersampling_strategy[i] = CLASS_BALANCE
    elif count_train[i] <= CLASS_BALANCE:
        oversampling_strategy[i] = CLASS_BALANCE
print(f"Trainset undersampling_strategy: {undersampling_strategy}")
print(f"Trainset oversampling_strategy: {oversampling_strategy}")

under = RandomUnderSampler(sampling_strategy=undersampling_strategy)
over = RandomOverSampler(sampling_strategy=oversampling_strategy)
steps = [("u", under), ("o", over)]
pipeline = Pipeline(steps=steps)
Z_train_balanced, Y_train_balanced = pipeline.fit_resample(Z_train, Y_train)

count_train_balanced = Counter(Y_train_balanced)

# Testset

count_test = Counter(Y_test)

# min_key = min(count_test, key=count_test.get)
# min_value = count_test[min_key]

undersampling_strategy = {}
for i in count_test:
    if count_test[i] > TEST_BALANCE:
        undersampling_strategy[i] = TEST_BALANCE

print(f"Testset undersampling_strategy: {undersampling_strategy}")

under = RandomUnderSampler(sampling_strategy=undersampling_strategy)
steps = [("u", under)]
pipeline = Pipeline(steps=steps)
Z_test_balanced, Y_test_balanced = pipeline.fit_resample(Z_test, Y_test)

count_test_balanced = Counter(Y_test_balanced)

print(f"Trainset: {count_train}")
print(f"Testset: {count_test}")
print(f"Trainset balanced: {count_train_balanced}")
print(f"Testset balanced: {count_test_balanced}")

In [ ]:
for index, z in enumerate(Z_train):
    Z_train[index] = Z_train[index][0]
for index, z in enumerate(Z_test):
    Z_test[index] = Z_test[index][0]
for index, z in enumerate(Z_train_balanced):
    Z_train_balanced[index] = Z_train_balanced[index][0]
for index, z in enumerate(Z_test_balanced):
    Z_test_balanced[index] = Z_test_balanced[index][0]

In [ ]:
# == Create binary_crossentropy X_train, Y_train, X_train_balanced, Y_train_balanced, X_test, Y_test, X_test_balanced, Y_test_balanced ==

for key in Y_dict:
    binary_crossentropy_labels = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for label in Y_dict[key]:
        binary_crossentropy_labels[label] = 1
    Y_dict[key] = binary_crossentropy_labels

X_train = []
Y_train = []
Z_train_new = []
X_train_balanced = []
Y_train_balanced = []
Z_train_balanced_new = []
X_test = []
Y_test = []
Z_test_new = []
X_test_balanced = []
Y_test_balanced = []
Z_test_balanced_new = []

for patient_id in set(Z_train):
    X_train.append(X_dict[patient_id])
    Y_train.append(Y_dict[patient_id])
    Z_train_new.append(patient_id)

for patient_id in set(Z_train_balanced):
    X_train_balanced.append(X_dict[patient_id])
    Y_train_balanced.append(Y_dict[patient_id])
    Z_train_balanced_new.append(patient_id)

for patient_id in set(Z_test):
    X_test.append(X_dict[patient_id])
    Y_test.append(Y_dict[patient_id])
    Z_test_new.append(patient_id)

for patient_id in set(Z_test_balanced):
    X_test_balanced.append(X_dict[patient_id])
    Y_test_balanced.append(Y_dict[patient_id])
    Z_test_balanced_new.append(patient_id)

Z_train = Z_train_new
Z_train_balanced = Z_train_balanced_new
Z_test = Z_test_new
Z_test_balanced_new = Z_test_balanced_new

In [ ]:
# == Shuffle data, convert to numpy lists and reshape ==

# Shuffle data

combined = list(zip(X_train, Y_train, Z_train))
random.shuffle(combined)
X_train, Y_train, Z_train = zip(*combined)
X_train = list(X_train)
Y_train = list(Y_train)
Z_train = list(Z_train)

combined = list(zip(X_test, Y_test, Z_test))
random.shuffle(combined)
X_test, Y_test, Z_test = zip(*combined)
X_test = list(X_test)
Y_test = list(Y_test)
Z_test = list(Z_test)

combined = list(zip(X_train_balanced, Y_train_balanced, Z_train_balanced))
random.shuffle(combined)
X_train_balanced, Y_train_balanced, Z_train_balanced = zip(*combined)
X_train_balanced = list(X_train_balanced)
Y_train_balanced = list(Y_train_balanced)
Z_train_balanced = list(Z_train_balanced)

combined = list(zip(X_test_balanced, Y_test_balanced, Z_test_balanced))
random.shuffle(combined)
X_test_balanced, Y_test_balanced, Z_test_balanced = zip(*combined)
X_test_balanced = list(X_test_balanced)
Y_test_balanced = list(Y_test_balanced)
Z_test_balanced = list(Z_test_balanced)

print(f"Y_train: {Y_train[:50]}")
print(f"Y_test: {Y_test[:50]}")
print(f"Y_train_balanced: {Y_train_balanced[:50]}")
print(f"Y_test_balanced: {Y_test_balanced[:50]}")

# Convert to numpy lists

# Y_train = np.array(Y_train)
# Y_test = np.array(Y_test)
# Y_train_balanced = np.array(Y_train_balanced)
# Y_test_balanced = np.array(Y_test_balanced)

for index, x in enumerate(Y_train):
    Y_train[index] = np.array(x)
Y_train = np.array(Y_train)

for index, x in enumerate(Y_test):
    Y_test[index] = np.array(x)
Y_test = np.array(Y_test)

for index, x in enumerate(Y_train_balanced):
    Y_train_balanced[index] = np.array(x)
Y_train_balanced = np.array(Y_train_balanced)

for index, x in enumerate(Y_test_balanced):
    Y_test_balanced[index] = np.array(x)
Y_test_balanced = np.array(Y_test_balanced)

for index, x in enumerate(X_train):
    X_train[index] = np.array(x)
X_train = np.array(X_train)

for index, x in enumerate(X_test):
    X_test[index] = np.array(x)
X_test = np.array(X_test)

for index, x in enumerate(X_train_balanced):
    X_train_balanced[index] = np.array(x)
X_train_balanced = np.array(X_train_balanced)

for index, x in enumerate(X_test_balanced):
    X_test_balanced[index] = np.array(x)
X_test_balanced = np.array(X_test_balanced)

# Reshape

X_train = X_train.reshape((-1, 2000, 1))
X_test = X_test.reshape((-1, 2000, 1))
X_train_balanced = X_train_balanced.reshape((-1, 2000, 1))
X_test_balanced = X_test_balanced.reshape((-1, 2000, 1))

In [ ]:
# == A-B-testing models ==

# Model A: Residual_CNN_1lead
def conv(i, filters=16, kernel_size=3, strides=1):
    i = keras.layers.Conv1D(
        filters=filters, kernel_size=kernel_size, strides=strides, padding="same"
    )(i)
    i = keras.layers.BatchNormalization()(i)
    i = keras.layers.LeakyReLU()(i)
    i = keras.layers.SpatialDropout1D(0.1)(i)
    return i
def residual_unit(x, filters, layers=3):
    inp = x
    for i in range(layers):
        x = conv(x, filters)
    return keras.layers.add([x, inp])
def conv_block(x, filters, strides):
    x = conv(x, filters)
    x = residual_unit(x, filters)
    if strides > 1:
        x = keras.layers.AveragePooling1D(strides, strides)(x)
    return x
def build_model_A(input_shape, num_classes):
    inp = keras.layers.Input(input_shape)
    x = inp
    x = conv_block(x, 16, 2)
    x = conv_block(x, 32, 2)
    x = conv_block(x, 64, 2)
    x = conv_block(x, 128, 2)
    # x = keras.layers.Masking(mask_value=0)(x)
    # x = keras.layers.GRU(128, recurrent_dropout=0.1)(x)
    x = keras.layers.GlobalAveragePooling1D()(x)
    x = keras.layers.Dense(num_classes, activation="sigmoid")(x)
    model = keras.models.Model(inp, x)
    return model

# Model B: CNN_Transformer_1lead
def build_model_B(num_classes, input_shape):
    # input_shape = (2000, 1)  # Each sample has 2000 timesteps and 1 feature per timestep
    input_layer = keras.layers.Input(input_shape)

    # Masking for padded/truncated data
    # i = keras.layers.Masking(mask_value=0)(input_layer)
    # Conv1
    i = keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, padding="same")(input_layer)
    i = keras.layers.BatchNormalization()(i)
    i = keras.layers.ReLU()(i)
    i = keras.layers.SpatialDropout1D(0.1)(i)
    i = keras.layers.AveragePooling1D(2)(i)
    # Conv2
    i = keras.layers.Conv1D(filters=256, kernel_size=3, strides=1, padding="same")(i)
    i = keras.layers.BatchNormalization()(i)
    i = keras.layers.ReLU()(i)
    i = keras.layers.SpatialDropout1D(0.1)(i)
    i = keras.layers.AveragePooling1D(2)(i)
    # Conv3
    i = keras.layers.Conv1D(filters=512, kernel_size=3, strides=1, padding="same")(i)
    i = keras.layers.BatchNormalization()(i)
    i = keras.layers.ReLU()(i)
    i = keras.layers.SpatialDropout1D(0.1)(i)
    i = keras.layers.AveragePooling1D(2)(i)
    # Conv3
    i = keras.layers.Conv1D(filters=1024, kernel_size=3, strides=1, padding="same")(i)
    i = keras.layers.BatchNormalization()(i)
    i = keras.layers.ReLU()(i)
    i = keras.layers.SpatialDropout1D(0.1)(i)
    i = keras.layers.AveragePooling1D(2)(i)
    # Channel Average Pooling and Reshaping
    i = keras.layers.GlobalAveragePooling1D(data_format="channels_last")(i)
    i = keras.layers.Reshape((8, 128))(i)
    # Encoder block/Attention mechanisms
    i = transformer_encoder(i, input_shape=(8, 128), key_dim=8, num_heads=8, ff_dim=200, dropout=0.1)
    i = transformer_encoder(i, input_shape=(8, 128), key_dim=8, num_heads=8, ff_dim=200, dropout=0.1)
    i = transformer_encoder(i, input_shape=(8, 128), key_dim=8, num_heads=8, ff_dim=200, dropout=0.1)
    # Flatten
    i = keras.layers.Flatten()(i)
    # Feedforward Softmax
    i = keras.layers.Dense(num_classes, activation="sigmoid")(i)
    return keras.models.Model(inputs=input_layer, outputs=i)

# Model C: vanilla_Transformer_1lead
def transformer_encoder(input, input_shape, num_heads, key_dim, ff_dim, dropout):
    # Multi-Head Attention
    x = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim, dropout=dropout, kernel_regularizer=regularizers.l2(0.001))(input, input)
    # Add & Normalize
    res = x + input
    x = keras.layers.LayerNormalization(epsilon=1e-6)(res)
    # Feed-Forward Layer
    x = keras.layers.Flatten(input_shape=input_shape)(x)
    x = keras.layers.Dense(units=ff_dim, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = keras.layers.Dense(input_shape[0] * input_shape[1], kernel_regularizer=regularizers.l2(0.001))(x)
    x = keras.layers.Reshape(input_shape)(x)
    x = keras.layers.Dropout(rate=dropout)(x)
    # Add & Normalize
    x = x + res
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    return x

def build_model_C(num_classes, input_shape, num_encoder_blocks, num_heads, key_dim, ff_dim, dropout):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_encoder_blocks):
        x = transformer_encoder(x, input_shape, key_dim, num_heads, ff_dim, dropout)
    x = keras.layers.Flatten(input_shape=input_shape)(x)
    outputs = keras.layers.Dense(num_classes, activation='sigmoid')(x)
    return keras.Model(inputs, outputs)

# Model D: channel_attention_1lead
from tensorflow.keras.layers import Input, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Dense, Multiply, Add, Activation
from tensorflow.keras.models import Model

def channel_attention(input_feature, ratio=8):
    channel = input_feature.shape[-1]

    shared_layer_one = Dense(channel//ratio, activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
    shared_layer_two = Dense(channel, kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')

    avg_pool = GlobalAveragePooling1D()(input_feature)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    max_pool = GlobalMaxPooling1D()(input_feature)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)

    cbam_feature = Add()([avg_pool, max_pool])
    cbam_feature = Activation('sigmoid')(cbam_feature)

    return Multiply()([input_feature, cbam_feature])

def build_model_D(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    x = Conv1D(64, kernel_size=3, padding='same', activation='relu')(inputs)
    x = channel_attention(x)

    x = Conv1D(128, kernel_size=3, padding='same', activation='relu')(x)
    x = channel_attention(x)

    x = Conv1D(256, kernel_size=3, padding='same', activation='relu')(x)
    x = channel_attention(x)

    x = GlobalAveragePooling1D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

In [ ]:
def conv(i, filters=16, kernel_size=3, strides=1):
    i = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding="same")(i)
    i = keras.layers.BatchNormalization()(i)
    i = keras.layers.LeakyReLU()(i)
    i = keras.layers.SpatialDropout1D(0.1)(i)
    return i

def inception(inputs, filters):
    conv1 = conv(inputs, filters, kernel_size=3)
    conv2 = conv(inputs, filters, kernel_size=5)
    # conv3 = conv(inputs, filters, kernel_size=7)
    conv4 = conv(inputs, filters, kernel_size=9)
    # conv5 = conv(inputs, filters, kernel_size=11)
    # conv6 = conv(inputs, filters, kernel_size=13)
    conv7 = conv(inputs, filters, kernel_size=15)
    # concatenated = keras.layers.Concatenate()([conv1, conv2, conv3, conv4, conv5, conv6, conv7])
    concatenated = keras.layers.Concatenate()([conv1, conv2, conv4, conv7])
    return concatenated

def residual_unit(x, filters, layers=1):
    inp = x
    for i in range(layers):
        x = inception(x, filters)
        # x = conv(x, filters)
    return keras.layers.add([x, inp])

def conv_block(x, filters, strides):
    x = conv(x, filters*4)
    x = residual_unit(x, filters)
    if strides > 1:
        x = keras.layers.MaxPooling1D(strides, strides)(x)
    return x

def build_model(input_shape, num_classes):
    inp = keras.layers.Input(input_shape)
    x = inp
    x = conv_block(x, 16, 2)
    x = conv_block(x, 32, 2)
    # x = conv_block(x, 64, 2)
    # x = conv_block(x, 128, 2)
    x = keras.layers.GlobalMaxPooling1D()(x)
    x = keras.layers.Dense(num_classes, activation="sigmoid")(x)
    model = keras.models.Model(inp, x)
    return model

In [ ]:
# == Initialize model ==

# Explicitly specify the GPU device
physical_devices = tf.config.experimental.list_physical_devices("GPU")
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

input_shape = X_train_balanced.shape
num_encoder_blocks = 8
num_heads = 8 # 8
key_dim = 25 # 25
ff_dim = 24 # 24
dropout = 0.1

# Reshape
X_train = X_train.reshape(-1, 2000, 1) # (-1, 2000, 1) (-1, 10, 200) (-1, 40, 50)
X_test = X_test.reshape(-1, 2000, 1)
X_train_balanced = X_train_balanced.reshape(-1, 2000, 1)
X_test_balanced = X_test_balanced.reshape(-1, 2000, 1)

# model = build_model_D(num_classes=NUM_CLASSES, input_shape=input_shape[1:], num_encoder_blocks=num_encoder_blocks, num_heads=num_heads, key_dim=key_dim, ff_dim=ff_dim, dropout=dropout)
model = build_model(num_classes=NUM_CLASSES, input_shape=input_shape[1:])

model.summary()

In [ ]:
# Load the class weights from the uploaded file
weights_path = os.path.join(path, "prepared/Physionet2021_evaluation/weights.csv")
class_weights_df = pd.read_csv(weights_path)

# Reorder rows according to new_order
new_order = [1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 17, 13, 18, 14, 0, 3, 15, 16, 4, 19, 20, 21, 22, 23, 24, 25]
class_weights_df = class_weights_df.iloc[new_order].reset_index(drop=True)


# Set the index and convert to numpy array
class_weights = class_weights_df.set_index(class_weights_df.columns[0]).values

# Calculate the inverse of class weights
inverse_class_weights = 1 / class_weights

def binary_cross_entropy(y_true, y_pred):
    return tf.reduce_mean(tf.keras.losses.binary_crossentropy(y_true, y_pred))

def sparsity_loss(y_pred):
    return -4 * y_pred * (y_pred - 1)

def challenge_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # Compute confusion matrix entries
    a_ij = tf.matmul(tf.expand_dims(y_true, -1), tf.expand_dims(y_pred, 1))

    # Compute weighted sum
    cl_loss = tf.reduce_sum(class_weights * a_ij)
    return cl_loss

# Custom loss function
def custom_loss(y_true, y_pred):
    bce = binary_cross_entropy(y_true, y_pred)
    sl = sparsity_loss(y_pred)
    cl = challenge_loss(y_true, y_pred)

    return bce - cl + sl

In [ ]:
# == Train model ==

EPOCHS = 50
BATCH_SIZE = 32

# Check if a GPU is available
print("Number of GPUs available:", len(tf.config.experimental.list_physical_devices("GPU")))
print(f"Number of training data examples: {len(X_train_balanced)}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Shape of training data: {input_shape}")
print(f"Epochs: {EPOCHS}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Batch size: {BATCH_SIZE}")

callbacks = [
    keras.callbacks.ModelCheckpoint(f"Model_{NUM_CLASSES}classes.h5", save_best_only=True, monitor="val_loss"),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=0.0000001),
    # reduce_lr,
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, verbose=1),
]

optimizer = Adam(learning_rate=LEARNING_RATE)
# optimizer = Adam(learning_rate=0.01)


# loss="binary_crossentropy", metrics=["binary_accuracy"]; sparse_categorical_crossentropy, metrics=["sparse_categorical_accuracy"]
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]) # "binary_crossentropy" custom_loss
history = model.fit(X_train_balanced, Y_train_balanced, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=callbacks, validation_split=0.25, verbose=1)

train_accuracy = history.history["accuracy"] # list
val_accuracy = history.history["val_accuracy"] # list
train_loss = history.history["loss"] # list
val_loss = history.history["val_loss"] # list

In [ ]:
pred_prob = model.predict(X_train)

In [ ]:
pred_prob[1]
for i in pred_prob[1]:
    print(float(i))
print(Y_train[1])

In [ ]:
# Define the top layer
top_input = keras.Input(shape=(NUM_CLASSES,))
top_output = keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(top_input)
top_model = Model(inputs=top_input, outputs=top_output)

In [ ]:
# Load the class weights from the uploaded file
weights_path = os.path.join(path, "prepared/Physionet2021_evaluation/weights.csv")
class_weights_df = pd.read_csv(weights_path)

# Reorder rows according to new_order
new_order = [1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 17, 13, 18, 14, 0, 3, 15, 16, 4, 19, 20, 21, 22, 23, 24, 25]
class_weights_df = class_weights_df.iloc[new_order].reset_index(drop=True)
# Set the index and convert to numpy array
class_weights = class_weights_df.set_index(class_weights_df.columns[0]).values
# Calculate the inverse of class weights
inverse_class_weights = 1 / class_weights

def sparsity_loss(y_pred):
    return -4 * y_pred * (y_pred - 1)

def challenge_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # Compute confusion matrix entries
    a_ij = tf.matmul(tf.expand_dims(y_true, -1), tf.expand_dims(y_pred, 1))

    # Compute weighted sum
    cl_loss = tf.reduce_sum(class_weights * a_ij)
    return cl_loss

# Custom loss function
def custom_loss(y_true, y_pred):
    sl = sparsity_loss(y_pred)
    cl = challenge_loss(y_true, y_pred)

    return cl + sl

In [ ]:
# Compile the top model with custom loss
top_model.compile(optimizer='adam', loss=custom_loss)
# Train the top model using the output of the main model as input
top_model.fit(pred_prob, Y_train, epochs=5)

In [ ]:
# Get input of model A
inputs = model.input

# Use output from model A as input to model B
output_A = model(inputs)
output_B = top_model(output_A)

# Create a new model that links input from A to output of B
combined_model = tf.keras.Model(inputs=inputs, outputs=output_B)

In [ ]:
pred_prob2 = combined_model.predict(X_train)

In [ ]:
pred_prob2[1]
for i in pred_prob2[1]:
    print(float(i))
print(Y_train[1])

In [ ]:
pred_prob3 = combined_model.predict(X_test)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score

# Assuming 'predictions' from combined_model.predict(x_val) and 'y_val' are your validation labels
num_classes = 26  # Number of classes
thresholds = np.linspace(0, 1, 100)
optimal_thresholds = np.zeros(num_classes)

for i in range(num_classes):
    f1_scores = []
    for thresh in thresholds:
        # Apply threshold to predictions for class 'i'
        binary_predictions = (pred_prob[:, i] > thresh).astype(int)
        # Calculate F1 score
        f1 = f1_score(Y_test[:, i], binary_predictions)
        f1_scores.append(f1)

    # Find the threshold that maximized the F1 score for this class
    optimal_index = np.argmax(f1_scores)
    optimal_thresholds[i] = thresholds[optimal_index]
    print(f"Class {i} optimal threshold: {thresholds[optimal_index]} with F1 Score: {f1_scores[optimal_index]}")

In [ ]:
# == Plot training curve ==

# Accuracy

train_accuracy = train_accuracy
val_accuracy = val_accuracy
epochs_range = range(1, len(train_accuracy) + 1)
plt.figure(figsize=(12, 6))
plt.plot(epochs_range, train_accuracy, "r", label="Training Accuracy")
plt.plot(epochs_range, val_accuracy, "b", label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig(
    "Training_accuracy_"+str(NUM_CLASSES)+"classes.png",
    dpi=300,
    format="png",
    bbox_inches="tight",
)
plt.show()
plt.close()

# Loss

train_loss = train_loss
val_loss = val_loss
epochs_range = range(1, len(train_loss) + 1)
plt.figure(figsize=(12, 6))
plt.plot(epochs_range, train_loss, "r", label="Training Loss")
plt.plot(epochs_range, val_loss, "b", label="Validation Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.savefig(
    "Training_loss_"+str(NUM_CLASSES)+"classes.png",
    dpi=300,
    format="png",
    bbox_inches="tight",
)
plt.show()
plt.close()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Predict probabilities
pred_prob = model.predict(X_test)

# Binarize predictions using a threshold
threshold = 0.5
pred = (pred_prob > threshold).astype(int)

# Calculate accuracy and other metrics
accuracy = accuracy_score(Y_test, pred)
precision = precision_score(Y_test, pred, average='weighted')  # or 'macro'
recall = recall_score(Y_test, pred, average='weighted')
f1 = f1_score(Y_test, pred, average='weighted')

print(f"Accuracy test set: {accuracy}")
print(f"Precision test set: {precision}")
print(f"Recall test set: {recall}")
print(f"F1 Score test set: {f1}")

In [ ]:
# == Convert predictions unbalanced in Physionet
import shutil

try:
    shutil.rmtree("test_outputs")
except FileNotFoundError:
    print(f"Error: Directory not found.")
except PermissionError:
    print(f"Error: Permission denied.")
try:
    os.makedirs("test_outputs")
except OSError as e:
    print(f"Error: {e.strerror}")

pbar = tqdm(total=len(pred), desc="Convert test_outputs", position=0, leave=True)
for index, prediction in enumerate(tqdm(zip(pred, pred_prob))):
    pbar.update(1)
    new_file = "#"
    new_file += Z_test[index] + "\n"
    # ids
    for pred_index, _ in enumerate(prediction[0]):
        new_file += map_arrhyhtmia_index_to_id(pred_index) + ","
    new_file = new_file[:-1] + "\n"
    # pred
    for pred_index, _ in enumerate(prediction[0]):
        if prediction[0][pred_index] == 1:
            value = "True"
        elif prediction[0][pred_index] == 0:
            value = "False"
        new_file += value + ","
    new_file = new_file[:-1] + "\n"
    # pred_prob
    for pred_index, _ in enumerate(prediction[1]):
        new_file += str(prediction[1][pred_index]) + ","
    new_file = new_file[:-1]
    # with open(os.path.join(path, f"test_outputs/{Z_test[index]}.csv"), "w", encoding="utf-8") as file:
    #    file.write(new_file)
    with open(f"test_outputs/{Z_test[index]}.csv", "w", encoding="utf-8") as file:
        file.write(new_file)

In [ ]:
from google.colab import files
folder_to_zip = "test_outputs"
output_filename = 'test_outputs.zip'
shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_to_zip)
files.download(output_filename)

In [ ]:
# == Apply on testset balanced ==

# Predict probabilities
pred_prob = model.predict(X_test_balanced)

# Binarize predictions using a threshold
threshold = 0.5
pred = (pred_prob > threshold).astype(int)

# Calculate accuracy and other metrics
accuracy = accuracy_score(Y_test_balanced, pred)
precision = precision_score(Y_test_balanced, pred, average='micro')  # or 'macro'
recall = recall_score(Y_test_balanced, pred, average='micro')
f1 = f1_score(Y_test_balanced, pred, average='micro')

print(f"Accuracy test set: {accuracy}")
print(f"Precision test set: {precision}")
print(f"Recall test set: {recall}")
print(f"F1 Score test set: {f1}")

In [ ]:
# == Plot trainset distribution ==

extracted_labels_testset = []
for label in Y_train:
    for index, _ in enumerate(label):
        if label[index]:
            extracted_labels_testset.append(index)

label_counts = Counter(extracted_labels_testset)
print(label_counts)
combined = list(zip(label_counts.keys(), label_counts.values()))
combined.sort(key=lambda x: x[1], reverse=True)
label_keys, label_values = zip(*combined)
label_keys = list(label_keys)
label_values = list(label_values)

label_keys = list(map(lambda x: str(x), label_keys))

# for index, key in enumerate(label_keys):
#    label_keys[index] = str(index + 1) + ". " + key[0].upper() + key[1:]

plt.figure(figsize=(30, 10))
plt.bar(label_keys, label_values, color="#1f77b4")
plt.title("Physionet 2021 labels")
plt.xlabel("Arrhythmia type", labelpad=7)
plt.ylabel("Occurence")
plt.xticks(rotation=45, ha="right", fontsize=16)  # (rotation='diagional')
bars = plt.bar(label_keys, label_values, color="#1f77b4")
# Adding the counts on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        yval + 5,
        yval,
        ha="center",
        va="bottom",
        fontsize=16,
    )

plt.show()
plt.close()

In [ ]:
# == Plot testset distribution ==

extracted_labels_testset = []
for label in Y_test:
    for index, _ in enumerate(label):
        if label[index]:
            extracted_labels_testset.append(index)

label_counts = Counter(extracted_labels_testset)
print(label_counts)
combined = list(zip(label_counts.keys(), label_counts.values()))
combined.sort(key=lambda x: x[1], reverse=True)
label_keys, label_values = zip(*combined)
label_keys = list(label_keys)
label_values = list(label_values)

label_keys = list(map(lambda x: str(x), label_keys))

# for index, key in enumerate(label_keys):
#    label_keys[index] = str(index + 1) + ". " + key[0].upper() + key[1:]

plt.figure(figsize=(30, 10))
plt.bar(label_keys, label_values, color="#1f77b4")
plt.title("Physionet 2021 labels")
plt.xlabel("Arrhythmia type", labelpad=7)
plt.ylabel("Occurence")
plt.xticks(rotation=45, ha="right", fontsize=16)  # (rotation='diagional')
bars = plt.bar(label_keys, label_values, color="#1f77b4")
# Adding the counts on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        yval + 5,
        yval,
        ha="center",
        va="bottom",
        fontsize=16,
    )

plt.show()
plt.close()